In [6]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time
import re

from bs4 import BeautifulSoup
from tqdm import tqdm
from warnings import filterwarnings
filterwarnings('ignore')


df = pd.read_csv('store_info_new.csv', encoding='euc-kr')
df['name_addr'] = df['store_name'].map(str)+" "+df['store_addr']
df['naver_map_url'] = '' # 미리 url을 담을 column을 만들어줌


driver = webdriver.Chrome(executable_path=r'C:\Users\quzmi\chromedriver.exe') # 웹드라이버가 설치된 경로를 지정해주시면 됩니다.

for i, keyword in enumerate(df['name_addr'].tolist()):
    print("이번에 찾을 키워드 :", i+1, f"/ {df.shape[0]} 행", keyword)
    
    try:
        naver_map_search_url = f'https://map.naver.com/v5/search/{keyword}/place' # 검색 url 만들기
        driver.get(naver_map_search_url) # 검색 url 접속, 즉 검색하기

        time.sleep(4) # 중요함
        cu = driver.current_url # 검색이 성공된 플레이스에 대한 개별 페이지

        res_code = re.findall(r"place/(\d+)", cu)
        final_url = 'https://pcmap.place.naver.com/restaurant/'+res_code[0]+'/review/visitor#'

        print(final_url)
        df['naver_map_url'][i]=final_url

    except:
        continue
#         keyword = df['store_name'].tolist()[i]
#         naver_map_search_url = f'https://map.naver.com/v5/search/{keyword}/place' # 검색 url 만들기
#         driver.get(naver_map_search_url) # 검색 url 접속, 즉 검색하기
        
#         df['naver_map_url'][i]= ''
# #         class_name = driver.find_element_by_class_name('._3Apve').getText()
        
# #         if keyword == class_name:
# #             class_name.click()

#         driver.find_element_by_xpath('//*[@id="_pcmap_list_scroll_container"]/ul/li[1]/div[2]/a[1]/div/div/span[1]').click()

#         time.sleep(4) # 중요함
#         cu = driver.current_url # 검색이 성공된 플레이스에 대한 개별 페이지

#         res_code = re.findall(r"place/(\d+)", cu)
#         final_url = 'https://pcmap.place.naver.com/restaurant/'+res_code[0]+'/review/visitor#'

#         print(final_url)
#         df['naver_map_url'][i]=final_url
#     #         print('none')

        
df.to_csv('url_completed.csv', encoding = 'utf-8-sig')


이번에 찾을 키워드 : 1 / 14 행 고성막국수 서울특별시 강서구 방화동 323-6
https://pcmap.place.naver.com/restaurant/12024487/review/visitor#
이번에 찾을 키워드 : 2 / 14 행 방화동쭈꾸미마을1997 서울특별시 강서구 방화동 451-8
https://pcmap.place.naver.com/restaurant/12824824/review/visitor#
이번에 찾을 키워드 : 3 / 14 행 공항칼국수 서울특별시 강서구 공항동 49-2
https://pcmap.place.naver.com/restaurant/19877546/review/visitor#
이번에 찾을 키워드 : 4 / 14 행 강서복집 서울특별시 강서구 등촌동 648
https://pcmap.place.naver.com/restaurant/11667758/review/visitor#
이번에 찾을 키워드 : 5 / 14 행 차돌집 서울특별시 강서구 방화동 611-35
https://pcmap.place.naver.com/restaurant/11727382/review/visitor#
이번에 찾을 키워드 : 6 / 14 행 오남매닭갈비 서울특별시 강서구 화곡동 110-65
https://pcmap.place.naver.com/restaurant/1422083628/review/visitor#
이번에 찾을 키워드 : 7 / 14 행 도일처 서울특별시 강서구 방화동 621-19
https://pcmap.place.naver.com/restaurant/13574886/review/visitor#
이번에 찾을 키워드 : 8 / 14 행 원조나주곰탕 서울특별시 강서구 방화동 559-1
https://pcmap.place.naver.com/restaurant/18411290/review/visitor#
이번에 찾을 키워드 : 9 / 14 행 마곡테이블 서울특별시 강서구 마곡동 796-5
https://pcmap.place.naver.com/rest

In [7]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time
import re

from bs4 import BeautifulSoup
from tqdm import tqdm

# 웹드라이버 접속
driver = webdriver.Chrome(executable_path=r'C:\Users\quzmi\chromedriver.exe')

df = pd.read_csv('url_completed.csv') #null값 처리 완료 데이터

# 수집할 정보들

# rating_list = [] # 평점
user_review_id = {} # 유저의 id - 추천시스템에서는 필수
review_json = {} # 리뷰
image_json = {} # 이미지

store_id_list = [] # 가게 아이디
portal_id_list = [] # 포털 아이디
date_list = [] # 리뷰 남긴 날짜
rating_list = [] # 평점
review_list = [] # csv에 저장할 리뷰 리스트

for j in range(len(df)):
    
    print('======================================================')
    print(str(j+1)+'번째 식당')
    
    # 식당 리뷰 개별 url 접속
    driver.get(df['naver_map_url'][j])
    thisurl = df['naver_map_url'][j]
    time.sleep(2)
    # 더보기 버튼 다 누르기
    # 더보기 버튼 누르기까지 10개
    # 더보기 버튼 누르면 10개 추가됨
#     while True:
#         try:
# #             time.sleep(1)
# #             driver.find_element_by_tag_name('body').send_keys(Keys.END)
# #             time.sleep(3)
# #             driver.find_element_by_css_selector('#app-root > div > div.place_detail_wrapper > div:nth-child(5) > div:nth-child(4) > div:nth-child(4) > div._2kAri > a').click()
#             driver.find_element_by_class_name('_3iTUo').click()
#             time.sleep(1.5)
# #             driver.find_element_by_tag_name('body').send_keys(Keys.END)
# #             time.sleep(1)
            
#         except NoSuchElementException:
#             print('-더보기 버튼 모두 클릭 완료-')
#             break

    
    # 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    time.sleep(1)
    
    # 식당 구분
    restaurant_name = df['store_name'][j]
    print('식당 이름 : '+restaurant_name)
    
    user_review_id[restaurant_name] = {}
    review_json[restaurant_name] = {}
    image_json[restaurant_name] = {}
    
    try:
        restaurant_classificaton = soup.find_all('span',attrs = {'class':'_3ocDE'})[0].text
    except:
        restaurant_classificaton = 'none'
    print('식당 구분 : '+restaurant_classificaton)
    print('----------------------------------------------')
    
    try:
        one_review = soup.find_all('div', attrs = {'class':'_1Z_GL'})
        review_num = len(one_review) # 특정 식당의 리뷰 총 개수
        print('리뷰 총 개수 : '+str(review_num))
    
        # 리뷰 개수
        for i in range(len(one_review)):
        
            # user url
            user_url = one_review[i].find('div', attrs = {'class':'_23Rml'}).find('a').get('href')
            print('user_url = '+user_url)
            
            # user url로부터 user code 뽑아내기
            user_code = re.findall(r"my/(\w+)", user_url)[0]
            print('user_code = '+user_code)
            
            # review 1개에 대한 id 만들기 
            res_code = re.findall(r"restaurant/(\d+)", thisurl)[0]
            review_id = str(res_code)+"_"+user_code
            print('review_id = '+review_id)
            
            # rating, 별점
            rating = one_review[i].find('span', attrs = {'class':'_2tObC'}).text
            print('rating = '+rating)
            
            # 주의!!! 사진 리뷰 유무에 따라 날짜 파싱코드 다름
            # ('span', attrs = {'class':'_3WqoL'})
            # 사진 없는 경우 : 총 6개 중 4번째
            # 사진 있는 경우 : 총 5개 중 3번째
            
            # date
            # 사진 리뷰 없음
            if len(one_review[i].find_all('span', attrs = {'class':'_3WqoL'})) == 5:
                date = one_review[i].find_all('span', attrs = {'class':'_3WqoL'})[2].text
            elif len(one_review[i].find_all('span', attrs = {'class':'_3WqoL'})) == 6:
                date = one_review[i].find_all('span', attrs = {'class':'_3WqoL'})[3].text
            else:
                date = ""
            print('date = '+date)
            
            # review 내용
            try : 
                review_content = one_review[i].find('span', attrs = {'class':'WoYOw'}).text
            except: # 리뷰가 없다면
                review_content = ""
            print('리뷰 내용 : '+review_content)
            
            
            # image 내용
            sliced_soup = one_review[i].find('div', attrs = {'class':'_1aFEL _2GO1Q'})

            if (sliced_soup != None):
                sliced_soup = sliced_soup.find('div',attrs={'class':'dRZ2X'})
                
                try:
                    img_url = 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=l&size=800x800&src='+re.findall(r'src=(.*jpeg)', str(sliced_soup))[0]
                except :
                    if (len(re.findall(r'src=(.*jpg)', str(sliced_soup)))!= 0):
                        img_url = 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=l&size=800x800&src='+re.findall(r'src=(.*jpg)', str(sliced_soup))[0]
                    elif (len(re.findall(r'src=(.*png)', str(sliced_soup)))!= 0):
                        img_url = 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=l&size=800x800&src='+re.findall(r'src=(.*png)', str(sliced_soup))[0]
                    else :
                        img_url = ""
            else:
                img_url = ""
                
            print('이미지 url : '+img_url)
            print('----------------------------------------------')
            print('\n')

            
            #리뷰정보
            # user_review_id
            user_review_id[restaurant_name][user_code] = review_id
            
            # review_json
            review_json[restaurant_name][review_id] = review_content
            
            # image_json
            image_json[restaurant_name][review_id] = img_url
            
            # rating_df_list
            naver_review = user_code, restaurant_name, rating, date
#             rating_list.append(naver_review)
            
            # csv list
            store_id_list.append(j + 1)
            portal_id_list.append("1004")
            date_list.append(date)
            rating_list.append(rating)
            review_list.append(review_content)
            
            
    
    
    except NoSuchElementException:
            none_review = "네이버 리뷰 없음"
            print(none_review)
            review_num = 0
            #리뷰정보 = restaurant_name, restaurant_classification, review_num, none_review
            
            # rating_df_list
            naver_review = user_code, restaurant_name, none_review, none_reivew
#             rating_list.append(naver_review)
            
            
    print('\n')
data_frame = pd.DataFrame()
data_frame['store_id'] = store_id_list
data_frame['portal_id'] = portal_id_list
data_frame["date"] = date_list
data_frame['score'] = rating_list
data_frame['review'] = review_list

data_frame.to_csv('store_review_homework.csv', encoding = 'utf-8-sig')

1번째 식당
식당 이름 : 고성막국수
식당 구분 : 막국수
----------------------------------------------
리뷰 총 개수 : 10
user_url = https://m.place.naver.com/my/5f04d2eb06ebc43aba7e09a4/review
user_code = 5f04d2eb06ebc43aba7e09a4
review_id = 12024487_5f04d2eb06ebc43aba7e09a4
rating = 4
date = 2021.10.11
리뷰 내용 : 좋아요
이미지 url : 
----------------------------------------------


user_url = https://m.place.naver.com/my/5c7425a70c90ccd7c55a1343/review
user_code = 5c7425a70c90ccd7c55a1343
review_id = 12024487_5c7425a70c90ccd7c55a1343
rating = 0.5
date = 2021.10.04
리뷰 내용 : 오픈하자마자 가서 주문하고 기다리는데 나중에 온 손님들 먼저 주시더라구요 그렇게 30분 기다리다가 뒷손님들 한테이블도 아니고 저희를 제외한 나
머지 테이블은 다 드시고 계시더라구요 아무리 그래도 이건 아니다 싶어서 저희가 제일 먼저 왔는데 왜 안주시냐고 물었더니 쓱 한번 쳐다보시고 다시 
들어가시더라구요? 그러고 또 15분 지나고 말씀드렸더니 저희 테이블에 놓여져있는 주문서를 보시고 이게 왜 여기있지? 하시더니 주방에 들어...
이미지 url : https://search.pstatic.net/common/?autoRotate=true&quality=95&type=l&size=800x800&src=https%3A%2F%2Fmyplace-phinf.pstatic.net%2F20211004_36%2F1633319903441J1How_JPEG%2Fupload_a32804c57f958ed778299850ff78de

In [ ]:
open_csv = pd.read_csv('store_review_homeword')